<a href="https://colab.research.google.com/github/stasiek-j/SAD2/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import distributions
import sklearn, torch
torch.manual_seed(42)

In [2]:
K = 3 # Ilość klastrów
N = 2 # Ilu wymiarowe dane 
size = (K, N)

In [3]:
component = distributions.Independent(distributions.Normal(
    torch.randn(size), torch.rand(size)), 1)
mixture = distributions.Categorical(torch.ones(K,))

gmm = distributions.MixtureSameFamily(mixture, component)

In [4]:
gmm.sample()

tensor([0.8017, 0.5883])

In [5]:
d = gmm.sample((10,))
print(d)
d.shape

tensor([[-0.0876,  0.8052],
        [-2.1717,  0.6077],
        [-0.5629,  0.4603],
        [ 0.3422,  0.3295],
        [ 0.8366, -0.2355],
        [-0.1124, -0.1567],
        [ 0.9697, -0.2746],
        [ 1.4971,  0.4033],
        [ 0.9680,  0.1806],
        [ 0.4519,  0.5627]])


torch.Size([10, 2])

In [6]:
gmm.component_distribution

Independent(Normal(loc: torch.Size([3, 2]), scale: torch.Size([3, 2])), 1)

In [7]:
parameters = [(torch.Tensor([.5, .5]), torch.Tensor([[.5, 0], [0, .5]])),
              (torch.Tensor([.4, .4]), torch.Tensor([[.2, 0], [0, .2]]))]

In [8]:
def prob(x, params):
  dis = distributions.MultivariateNormal(*params)
  return dis.log_prob(x)

In [9]:
def e_step(data, params):
  for i in range(data.shape[0]):
    x = data[i, 0]
    y = data[i, 1]
    pcluster1 = prob(torch.Tensor((x, y)), params[0])
    pcluster2 = prob(torch.Tensor((x, y)), params[1])
    print(pcluster1, pcluster2)

  pass


def m_step():
  pass

In [10]:
e_step(d, parameters)

tensor(-1.5831) tensor(-1.2332)
tensor(-8.2943) tensor(-16.8703)
tensor(-2.2760) tensor(-2.5553)
tensor(-1.1987) tensor(-0.2492)
tensor(-1.7990) tensor(-1.7147)
tensor(-1.9510) tensor(-1.6596)
tensor(-1.9654) tensor(-2.1777)
tensor(-2.1483) tensor(-3.2375)
tensor(-1.4658) tensor(-1.1555)
tensor(-1.1510) tensor(-0.3013)
